## 1. Deep Q-Network (DQN)

Deep Q-Network (DQN)**은 딥 강화 학습(Deep Reinforcement Learning)에서 자주 사용되는 알고리즘으로, Q-러닝과 신경망을 결합하여 복잡한 환경에서 학습할 수 있도록 합니다. DQN은 특히 게임 AI나 자동화된 트레이딩 시스템에서 널리 사용됩니다.

Deep Q-Network (DQN) 개요
    
목표: 에이전트가 환경과 상호작용하면서, 최적의 행동을 학습해 누적 보상을 최대화하는 것입니다.

Q-러닝: 𝑄(𝑠,𝑎)는 특정 상태 
    
𝑠에서 행동 𝑎를 취했을 때 예상되는 미래 보상입니다.
    
DQN 구조: 신경망을 사용하여 𝑄 함수를 근사화하며, 이를 통해 상태-행동 가치 함수를 학습합니다.

### 1) 자동화된 트레이딩 시스템 사례 (삼성전자)

In [ ]:
# 코드 설명
# 삼성전자 주식 데이터:

# yfinance 라이브러리를 사용해 삼성전자의 주식 데이터를 다운로드하고, 수익률(Return)을 계산합니다.
# StockTradingEnv 클래스:

# OpenAI Gym 스타일의 환경을 정의하여, 에이전트가 주식을 매수, 매도, 보유할 수 있도록 합니다.
# DQN 에이전트:

# 신경망을 사용해 Q-함수를 근사화하고, 탐험-탐사 균형을 조절합니다 (epsilon-greedy 전략).
# 학습 및 평가:

# 에이전트는 여러 에피소드를 통해 학습하고, 점점 더 높은 보상을 목표로 행동을 최적화합니다.
# 결과 시각화:

# 삼성전자 주식의 종가 그래프를 시각화합니다.


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
import gym
from gym import spaces
import matplotlib.pyplot as plt

# FutureWarning 무시
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. 삼성전자 주식 데이터 로드
df = yf.download("005930.KS", start="2020-01-01", end="2023-01-01")

# 인덱스의 날짜를 UTC로 변환하여 tz_localize 에러 해결
df.index = pd.to_datetime(df.index, utc=True)

# 결측값 처리
df["Dividends"] = df["Dividends"].fillna(0)
df["Stock Splits"] = df["Stock Splits"].fillna(0)
df['Return'] = df['Close'].pct_change()
df = df.dropna()
print(df.head())

# 2. 트레이딩 환경 정의
class StockTradingEnv(gym.Env):
    def __init__(self, data):
        super(StockTradingEnv, self).__init__()
        self.data = data
        self.n_step = len(data)
        self.current_step = 0
        self.balance = 1000000  # 초기 자본 (1,000,000 원)
        self.shares = 0  # 보유 주식 수
        self.total_profit = 0
        self.action_space = spaces.Discrete(3)  # [0: 보유, 1: 매수, 2: 매도]
        self.observation_space = spaces.Box(low=0, high=1, shape=(5,), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        self.balance = 1000000
        self.shares = 0
        self.total_profit = 0
        return self._next_observation()

    def _next_observation(self):
        obs = np.array([
            self.data.iloc[self.current_step]['Close'],
            self.data.iloc[self.current_step]['Return'],
            self.balance,
            self.shares,
            self.total_profit
        ])
        return obs

    def step(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        reward = 0
        done = False

        # 행동: 0(보유), 1(매수), 2(매도)
        if action == 1:  # 매수
            if self.balance >= current_price:
                self.shares += 1
                self.balance -= current_price
        elif action == 2:  # 매도
            if self.shares > 0:
                self.shares -= 1
                self.balance += current_price
                reward = current_price  # 이익을 보상으로

        self.total_profit = self.balance + self.shares * current_price - 1000000
        self.current_step += 1

        if self.current_step >= self.n_step - 1:
            done = True

        obs = self._next_observation()
        return obs, reward, done, {}

# 3. DQN 모델 정의
def build_model(state_size, action_size):
    model = Sequential()
    model.add(Dense(24, input_dim=state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

# 4. DQN 에이전트 정의
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = build_model(state_size, action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# 5. 환경 설정 및 학습
env = StockTradingEnv(df)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
episodes = 100

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0
    done = False

    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

    agent.replay(32)
    print(f"Episode {e + 1}/{episodes}, Total Reward: {total_reward:.2f}, Epsilon: {agent.epsilon:.2f}")

# 6. 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'], label='Samsung Electronics Close Price')
plt.title('Samsung Electronics Stock Trading with DQN')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()


결과 분석
학습이 진행됨에 따라, 에이전트는 점점 더 나은 매수/매도 전략을 학습하게 됩니다.
초기에는 무작위 탐험(epsilon=1.0)이 많지만, 시간이 지남에 따라 탐험이 줄고 탐사가 늘어납니다 (epsilon 감소).

### 2) KOSPI 데이터와 DQN

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
import gym
from gym import spaces
import matplotlib.pyplot as plt

# FutureWarning 무시
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. KOSPI 데이터 로드
df = yf.download("^KS11", start="2020-01-01", end="2023-01-01")

# 인덱스의 날짜를 UTC로 변환하여 tz_localize 에러 해결
df.index = pd.to_datetime(df.index, utc=True)

# 결측값 처리
df["Dividends"] = df["Dividends"].fillna(0)
df["Stock Splits"] = df["Stock Splits"].fillna(0)
df['Return'] = df['Close'].pct_change()
df = df.dropna()
print(df.head())

# 2. 트레이딩 환경 정의
class StockTradingEnv(gym.Env):
    def __init__(self, data):
        super(StockTradingEnv, self).__init__()
        self.data = data
        self.n_step = len(data)
        self.current_step = 0
        self.balance = 1000000  # 초기 자본 (1,000,000 원)
        self.shares = 0  # 보유 주식 수
        self.total_profit = 0
        self.action_space = spaces.Discrete(3)  # [0: 보유, 1: 매수, 2: 매도]
        self.observation_space = spaces.Box(low=0, high=1, shape=(5,), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        self.balance = 1000000
        self.shares = 0
        self.total_profit = 0
        return self._next_observation()

    def _next_observation(self):
        obs = np.array([
            self.data.iloc[self.current_step]['Close'],
            self.data.iloc[self.current_step]['Return'],
            self.balance,
            self.shares,
            self.total_profit
        ])
        return obs

    def step(self, action):
        current_price = self.data.iloc[self.current_step]['Close']
        reward = 0
        done = False

        # 행동: 0(보유), 1(매수), 2(매도)
        if action == 1:  # 매수
            if self.balance >= current_price:
                self.shares += 1
                self.balance -= current_price
        elif action == 2:  # 매도
            if self.shares > 0:
                self.shares -= 1
                self.balance += current_price
                reward = current_price  # 이익을 보상으로

        self.total_profit = self.balance + self.shares * current_price - 1000000
        self.current_step += 1

        if self.current_step >= self.n_step - 1:
            done = True

        obs = self._next_observation()
        return obs, reward, done, {}

# 3. DQN 모델 정의
def build_model(state_size, action_size):
    model = Sequential()
    model.add(Dense(24, input_dim=state_size, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(action_size, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
    return model

# 4. DQN 에이전트 정의
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = build_model(state_size, action_size)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, min(len(self.memory), batch_size))
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# 5. 환경 설정 및 학습
env = StockTradingEnv(df)
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)
episodes = 100

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0
    done = False

    while not done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward

    agent.replay(32)
    print(f"Episode {e + 1}/{episodes}, Total Reward: {total_reward:.2f}, Epsilon: {agent.epsilon:.2f}")

# 6. 결과 시각화
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Close'], label='KOSPI Close Price')
plt.title('KOSPI Index Trading with DQN')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()
